In [1]:
import geopandas as gpd
import pandas as pd
import os
from glob import glob
from sklearn.linear_model import LinearRegression
from tqdm.auto import tqdm
from tqdm.contrib.concurrent import process_map
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, root_mean_squared_error

In [2]:
cells = gpd.read_file('global-inputs/HR6-change-cells-aoi.gpkg')

cell_ids = cells.hex_id.tolist()

In [3]:
shoreline_files = pd.Series(sorted(glob("data/HR6/*/shoreline_timeseries_tidal_correction.csv")))

In [4]:
def calc_trends(file):
    df  = pd.read_csv(file)
    try:
        df.date = pd.to_datetime(df.date)
    except:
        print(file)
    df.index = (df.date - df.date.min()).dt.days / 365.25
    df.drop(columns=['date', 'Unnamed: 0', 'corrected_IW_shoreline_position', 'EOV_shoreline_chg'], inplace=True, errors='ignore')
    trends = {}
    for col in df.columns:
        sub_df = df[col].dropna()
        
        if not len(sub_df):
            continue
        x = sub_df.index.to_numpy().reshape(-1,1)
        y = sub_df
        linear_trend = LinearRegression().fit(x,y)
        pred = linear_trend.predict(x)

        trends['index'] = file.split('/')[2]
        if col == 'normalised_IW_shoreline_position':
            trends["IW shoreline trend"] = linear_trend.coef_[0]
            trends["IW shoreline intercept"] = linear_trend.intercept_
            trends["IW shoreline n_points"] = len(df[col])
            trends["IW shoreline n_points_nonan"] = len(sub_df)
            trends["IW shoreline r2_score"] = r2_score(y, pred)
            trends["IW shoreline mae"] = mean_absolute_error(y, pred)
            trends["IW shoreline mse"] = mean_squared_error(y, pred)
            trends["IW shoreline rmse"] = root_mean_squared_error(y, pred)
        else:
            trends["EOV shoreline trend"] = linear_trend.coef_[0]
            trends["EOV shoreline intercept"] = linear_trend.intercept_
            trends["EOV shoreline n_points"] = len(df[col])
            trends["EOV shoreline n_points_nonan"] = len(sub_df)
            trends["EOV shoreline r2_score"] = r2_score(y, pred)
            trends["EOV shoreline mae"] = mean_absolute_error(y, pred)
            trends["EOV shoreline mse"] = mean_squared_error(y, pred)
            trends["EOV shoreline rmse"] = root_mean_squared_error(y, pred)

    return pd.DataFrame([trends])
    

In [5]:
trends = pd.concat([calc_trends(f) for f in shoreline_files])

trends.dropna(inplace=True)

trends

/home/ubuntu/miniforge3/envs/coastal-monitoring/lib/python3.13/site-packages/sklearn/metrics/_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/ubuntu/miniforge3/envs/coastal-monitoring/lib/python3.13/site-packages/sklearn/metrics/_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


/home/ubuntu/miniforge3/envs/coastal-monitoring/lib/python3.13/site-packages/sklearn/metrics/_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/ubuntu/miniforge3/envs/coastal-monitoring/lib/python3.13/site-packages/sklearn/metrics/_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/ubuntu/miniforge3/envs/coastal-monitoring/lib/python3.13/site-packages/sklearn/metrics/_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


/home/ubuntu/miniforge3/envs/coastal-monitoring/lib/python3.13/site-packages/sklearn/metrics/_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/ubuntu/miniforge3/envs/coastal-monitoring/lib/python3.13/site-packages/sklearn/metrics/_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


/home/ubuntu/miniforge3/envs/coastal-monitoring/lib/python3.13/site-packages/sklearn/metrics/_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/ubuntu/miniforge3/envs/coastal-monitoring/lib/python3.13/site-packages/sklearn/metrics/_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


/home/ubuntu/miniforge3/envs/coastal-monitoring/lib/python3.13/site-packages/sklearn/metrics/_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


/home/ubuntu/miniforge3/envs/coastal-monitoring/lib/python3.13/site-packages/sklearn/metrics/_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


,index,IW shoreline trend,IW shoreline intercept,IW shoreline n_points,IW shoreline n_points_nonan,IW shoreline r2_score,IW shoreline mae,IW shoreline mse,IW shoreline rmse,EOV shoreline trend,EOV shoreline intercept,EOV shoreline n_points,EOV shoreline n_points_nonan,EOV shoreline r2_score,EOV shoreline mae,EOV shoreline mse,EOV shoreline rmse
0,86ba24017ffffff,0.702579,3.543091,168.0,168.0,0.050661,11.959151,191.941335,13.854289,-0.699641,3.159698,168.0,168.0,0.013339,23.332865,751.296405,27.409787
0,86ba24037ffffff,1.033395,-6.178385,165.0,165.0,0.344219,6.960677,67.921329,8.241440,1.584729,-20.738118,165.0,165.0,0.424481,5.610503,113.674259,10.661813
0,86ba24067ffffff,0.405417,15.691976,212.0,212.0,0.008782,10.310650,410.795336,20.268087,0.437827,-14.237220,212.0,212.0,0.004686,22.178457,901.716932,30.028602
0,86ba2414fffffff,0.348043,5.266681,184.0,184.0,0.017368,6.576343,103.833373,10.189866,0.527480,-14.009491,184.0,184.0,0.057555,7.288351,69.025181,8.308139
0,86ba2415fffffff,0.292272,2.335802,294.0,294.0,0.003130,14.973880,740.166095,27.205994,-0.346064,-9.880104,294.0,294.0,0.002264,27.429919,1435.851665,37.892633
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,86dab3707ffffff,0.835827,22.566646,230.0,230.0,0.075199,19.207722,498.013666,22.316220,0.180926,13.641783,230.0,230.0,0.072237,3.504994,24.369836,4.936581
0,86dab370fffffff,0.297456,27.512331,210.0,210.0,0.016397,15.723297,318.843525,17.856190,-0.194191,19.905761,210.0,210.0,0.005818,13.837996,387.106612,19.675025
0,86dab3737ffffff,1.939090,-13.700280,169.0,169.0,0.636444,7.864901,122.778906,11.080564,0.063353,-1.108063,169.0,169.0,0.039759,0.997845,5.541109,2.353956
0,86dab3757ffffff,-0.710451,22.826311,200.0,200.0,0.080666,15.351462,331.789675,18.215095,0.294009,10.300643,200.0,200.0,0.056095,7.670880,83.895965,9.159474


In [6]:
trends_file_path = 'shoreline_trends_by_cell.geojson'
# if os.path.exists(trends_file_path):
#     cell_trends = gpd.read_file('shoreline_trends_by_cell.geojson')
#     cell_trends.to_crs("EPSG:4326", inplace=True)
#     cell_trends.update(trends)
#     cell_trends.to_file(trends_file_path)
# else:
cell_trends = cells.merge(trends, on='index')
cell_trends.to_crs("EPSG:4326", inplace=True)
cell_trends.to_file(trends_file_path)
    
# cell_trends['hex_id'].to_csv("global-inputs/valid_cells.csv")

cell_trends

,index,hex_id,parent_id,sediment_area_ha,geometry,IW shoreline trend,IW shoreline intercept,IW shoreline n_points,IW shoreline n_points_nonan,IW shoreline r2_score,...,IW shoreline mse,IW shoreline rmse,EOV shoreline trend,EOV shoreline intercept,EOV shoreline n_points,EOV shoreline n_points_nonan,EOV shoreline r2_score,EOV shoreline mae,EOV shoreline mse,EOV shoreline rmse
0,86bb5e18fffffff,86bb5e18fffffff,85bb5e1bfffffff,16.76,"POLYGON ((173.94469 -35.12657, 173.94025 -35.0...",0.298079,-4.477878,132.0,132.0,0.029218,...,106.831661,10.335940,1.041199,-19.595942,132.0,132.0,0.042226,22.861959,889.839811,29.830183
1,86bb5e19fffffff,86bb5e19fffffff,85bb5e1bfffffff,20.00,"POLYGON ((174.01618 -35.12027, 174.01171 -35.0...",0.528597,1.945910,80.0,80.0,0.091311,...,58.638942,7.657607,0.433069,-3.924022,80.0,80.0,0.034089,9.785687,112.066496,10.586146
2,86bb58027ffffff,86bb58027ffffff,85bb5803fffffff,97.20,"POLYGON ((172.71095 -34.47891, 172.70704 -34.4...",0.826371,-0.681890,7.0,7.0,0.081622,...,29.049672,5.389775,3.016152,8.967772,7.0,7.0,0.070048,18.404964,456.612787,21.368500
3,86bb72b4fffffff,86bb72b4fffffff,85bb72b7fffffff,14.08,"POLYGON ((173.87492 -39.18614, 173.87027 -39.1...",-0.188351,23.975430,149.0,149.0,0.001322,...,643.891179,25.375011,1.961786,-60.850342,149.0,149.0,0.058567,33.639296,1485.815116,38.546272
4,86bb2bae7ffffff,86bb2bae7ffffff,85bb2baffffffff,44.20,"POLYGON ((175.14401 -40.80153, 175.13871 -40.7...",1.335347,3.361395,93.0,93.0,0.173572,...,448.711676,21.182816,0.940921,-8.912283,93.0,93.0,0.159426,10.672541,246.703678,15.706804
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
630,86da9285fffffff,86da9285fffffff,85da9287fffffff,2.00,"POLYGON ((171.98283 -41.59482, 171.97888 -41.5...",-0.275655,6.239350,250.0,250.0,0.010930,...,428.189401,20.692738,0.083857,10.765449,250.0,250.0,0.073839,1.316911,5.492715,2.343654
631,86da94ab7ffffff,86da94ab7ffffff,85da94abfffffff,50.36,"POLYGON ((172.82222 -43.2102, 172.81779 -43.17...",-0.168140,-10.409410,173.0,173.0,0.005401,...,346.140253,18.604845,0.319252,-2.556625,173.0,173.0,0.193476,4.298051,28.245895,5.314687
632,86da91d8fffffff,86da91d8fffffff,85da91dbfffffff,5.36,"POLYGON ((171.05489 -42.73298, 171.05131 -42.6...",-0.786736,11.915373,237.0,237.0,0.058619,...,437.476008,20.915927,0.091461,-0.328886,237.0,237.0,0.285459,0.812144,0.921556,0.959977
633,86bb299afffffff,86bb299afffffff,85bb299bfffffff,11.68,"POLYGON ((174.2819 -41.84778, 174.27691 -41.81...",2.875734,-40.942028,315.0,315.0,0.293260,...,691.488248,26.296164,-4.554379,22.297388,315.0,315.0,0.190290,43.047615,3062.329005,55.338314
